In [1]:
import pandas as pd
import numpy as np
from itertools import product

In [2]:
df = pd.read_csv("climate.csv")
array = np.array(df.iloc[:,:-1])
headers = df.columns
uniq = pd.Series({c:df[c].unique() for c in df})
for i in range(0,len(uniq) - 1):
    uniq[i] = uniq[i].tolist()
    uniq[i].append('?')

In [3]:
versionSpace = [list(x) for x in product(uniq[0], uniq[1], uniq[2], uniq[3], uniq[4], uniq[5])]
versionSpace.append(['0'] * (df.shape[1]-1))
attr = np.array(df.iloc[:,0:-1])
target = np.array(df.iloc[:,-1])

In [4]:
for i, x in enumerate(attr):
    if target[i] == 'Yes':
        to_remove = []
        for j, y in enumerate(versionSpace):
            for k in range(len(x)):
                if x[k] != y[k] and y[k] != '?':
                    to_remove.append(j)
                    break
        for j in sorted(to_remove, reverse=True):
            del versionSpace[j]

In [5]:
hypothesesList = pd.DataFrame(versionSpace)
hypothesesList.columns = headers[:-1]

print("Hypotheses: ")
print(hypothesesList)

Hypotheses: 
     sunny  warm normal  strong warm.1    same
0    sunny  warm   high  strong   warm    same
1    sunny  warm   high  strong   warm  change
2    sunny  warm   high  strong   warm       ?
3    sunny  warm   high  strong   cool    same
4    sunny  warm   high  strong   cool  change
..     ...   ...    ...     ...    ...     ...
320      ?     ?      ?       ?   cool       ?
321      ?     ?      ?       ?      ?    same
322      ?     ?      ?       ?      ?  change
323      ?     ?      ?       ?      ?       ?
324      0     0      0       0      0       0

[325 rows x 6 columns]


In [6]:
def isConsistent(example, hypothesis):
    for attribute, value in example.items():
        if hypothesis[attribute] != '?' and hypothesis[attribute] != value:
            return False
    return True

def filterConsistentHypotheses(hypothesesList, sampleData):
    consistentHypothesesDataframe = hypothesesList[hypothesesList.apply(lambda row: isConsistent(sampleData, row), axis=1)]
    return consistentHypothesesDataframe

In [7]:
consistent = hypothesesList
for i in array:
    dictionaryData = dict()
    for j, h in enumerate(headers[:-1]):
        dictionaryData[h] = i[j]
    consistent = filterConsistentHypotheses(consistent, dictionaryData)

print("Consistent Hypotheses:")
print(consistent)

Consistent Hypotheses:
    sunny warm normal  strong warm.1 same
296     ?    ?   high  strong      ?    ?
305     ?    ?   high       ?      ?    ?
314     ?    ?      ?  strong      ?    ?
323     ?    ?      ?       ?      ?    ?
